In [4]:
from PIL import Image
import glob
import numpy as np

In [5]:
def load_images(dir_name):
    output_images = []
    for image_name in glob.glob(dir_name + '/*'):
        image = Image.open(image_name).resize((224,224))
        output_image_data = np.asarray(image, dtype='float32') / 255.0
        if output_image_data.shape == (224,224,3):
            output_images.append(output_image_data)
    return np.array(output_images)

In [6]:
neutral_img = load_images('data/train/neutral')
happy_img = load_images('data/train/happy')
X = np.concatenate((neutral_img, happy_img), axis=0)

In [7]:
neutral_img.shape

(150, 224, 224, 3)

In [8]:
happy_img.shape

(150, 224, 224, 3)

In [9]:
import keras
num_classes = 2
y = np.concatenate((np.zeros(neutral_img.shape[0]), np.ones(happy_img.shape[0])))
y = keras.utils.to_categorical(y,num_classes)

/Users/cheunghoyeung/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [10]:
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=.2,random_state=42)

In [11]:
X_val.shape

(60, 224, 224, 3)

In [12]:
X_train.shape

(240, 224, 224, 3)

In [13]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras import backend as K

input_shape = (224, 224, 3)
num_classes = 2

model = Sequential([
    Conv2D(64, (3, 3), input_shape=input_shape, padding='same',activation='relu'),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    Conv2D(128, (3, 3), activation='relu', padding='same',),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    Conv2D(256, (3, 3), activation='relu', padding='same',),
    Conv2D(256, (3, 3), activation='relu', padding='same',),
    Conv2D(256, (3, 3), activation='relu', padding='same',),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    Conv2D(512, (3, 3), activation='relu', padding='same',),
    Conv2D(512, (3, 3), activation='relu', padding='same',),
    Conv2D(512, (3, 3), activation='relu', padding='same',),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    Conv2D(512, (3, 3), activation='relu', padding='same',),
    Conv2D(512, (3, 3), activation='relu', padding='same',),
    Conv2D(512, (3, 3), activation='relu', padding='same',),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    Flatten(),
    Dense(4096, activation='relu'),
    Dense(4096, activation='relu'),
    Dense(num_classes, activation='sigmoid')
])

In [16]:
from keras.optimizers import SGD
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=SGD(lr=0.1, momentum=0.9),
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train,y_train,epochs=5,batch_size=256,validation_data=(X_val,y_val))

Train on 240 samples, validate on 60 samples
Epoch 1/5
240/240 [==============================] - 1523s 6s/step - loss: 0.6932 - acc: 0.4875 - val_loss: 0.6931 - val_acc: 0.4833
Epoch 2/5


In [ ]:
model.save('vgg16_manual.h5')

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(16,8))

axes[0].plot(acc, 'bo', label='Training acc')
axes[0].plot(val_acc, 'b', label='Validation acc')
axes[0].set_title('Training and validation accuracy')
axes[0].legend()

axes[1].plot(loss, 'bo', label='Training loss')
axes[1].plot(val_loss, 'b', label='Validation loss')
axes[1].set_title('Training and validation loss')
axes[1].legend()

In [224]:
score, acc = model.evaluate(X_test, y_test)
print('Test score:', score)
print('Test accuracy:', acc)

16/16 [==============================] - 11s 667ms/step
Test score: 0.6955180764198303
Test accuracy: 0.5
